サーバレスコンピュートをお使いください

In [0]:
%run ./00_config

`/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/daily_weather_tokyo.csv`からread_filesを利用してcsvを読み込んで表示する。ただし、_rescued_dataは除外する。一行目は列名。

In [0]:
spark.sql(f"""
SELECT *
EXCEPT (_rescued_data)
FROM read_files(
  '/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/daily_weather_tokyo.csv',
  format => 'csv',
  header => 'true'
)
;
""")

上記をbronzeとして保存する

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE bronze_daily_weather_tokyo AS
SELECT *
EXCEPT (_rescued_data)
FROM read_files(
  '/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/daily_weather_tokyo.csv',
  format => 'csv',
  header => 'true'
);
""")

### Gold テーブルの作成
次にGold テーブルを作成します。<br>
Goldレイヤーでは、データの粒度を実際に分析する粒度に変換します。<br>
今回は月毎の平均値を使うことにします。<br><br>
※ 通常はSilverレイヤーで欠損値の処理などを行いますが、今回のハンズオンでは割愛します。

```
bronze_daily_weather_tokyo を月毎に集計して。
ave_tempは平均を、high_tempは最大値を、low_tempは最低値を求めて。小数点は二桁までにして。
```

In [0]:
spark.sql(f"""
SELECT
  date_format(date, 'yyyy-MM') AS month,
  ROUND(AVG(ave_temp), 2) AS avg_ave_temp,
  ROUND(MAX(high_temp), 2) AS max_high_temp,
  ROUND(MIN(low_temp), 2) AS min_low_temp
FROM bronze_daily_weather_tokyo
GROUP BY date_format(date, 'yyyy-MM')
ORDER BY month
;
""")

`上記をgoldテーブルとして保存して`

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE gold_monthly_weather_tokyo AS
SELECT
  date_format(date, 'yyyy-MM') AS month,
  ROUND(AVG(ave_temp), 2) AS avg_ave_temp,
  ROUND(MAX(high_temp), 2) AS max_high_temp,
  ROUND(MIN(low_temp), 2) AS min_low_temp
FROM bronze_daily_weather_tokyo
GROUP BY date_format(date, 'yyyy-MM')
ORDER BY month
;
""")